# Core: Exploring and understanding the Data

All imports are sorted and copy from the imports that I use in assignment2.

In [47]:
# importing pandas, and other necessary modules
import io
import sys

import matplotlib.image as mpimg
import matplotlib.pyplot as plt  # visualising
import missingno as msno
import numpy as np  # linear algebra
# import pandas, and other necessary modules
import pandas as pd  # data processing
# easy for structing the report , need pip install pandas-profiling first
import pandas_profiling as pp
import phik
import pydotplus
import seaborn as sns  # visualising
from IPython.display import Image
from numpy import cov
from scipy.stats import pearsonr
from sklearn import datasets, tree
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import (KBinsDiscretizer, LabelEncoder,
                                   OneHotEncoder, OrdinalEncoder,
                                   StandardScaler)
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

import glob


load dataset and merged them into 1 

In [48]:
alternative = pd.read_csv('datasets/alternative.csv')
anime = pd.read_csv("datasets/anime.csv")
blues = pd.read_csv('datasets/blues.csv')
classical = pd.read_csv('datasets/classical.csv')
country = pd.read_csv('datasets/country.csv')
electronics = pd.read_csv('datasets/electronic.csv')
hip_hop = pd.read_csv('datasets/hip-hop.csv')
jazz = pd.read_csv('datasets/jazz.csv')
rap = pd.read_csv('datasets/rap.csv')
rock = pd.read_csv('datasets/rock.csv')
test_instances = pd.read_csv('datasets/testing-instances.csv')

dataset_list = [
    alternative,
    anime,
    blues,
    classical,
    country,
    electronics,
    hip_hop,
    jazz,
    rap,
    rock
]

# dataset_name_list = ["alternative", "anime", "blues", "classical",
#                      "country", "electronics", "hip_hop", "jazz", "rap", "rock"]
# name_dataset_map = dict(zip(dataset_name_list, dataset_list))

# set the pd to display every rows and cols, so no folding
pd.set_option("display.max_rows", None, "display.max_columns", None)


def merge_multi_datasets(data_list=dataset_list):
    '''merge multiple dataset into one and then return it.
    For example, in data_list, it hold rock,rap ... etc, this method will merge it  using outer
    '''
    merged_all = pd.DataFrame()
    # print(merged_all)
    for dataset in data_list:
        # In order to merge the data without errors, we need to align the index of the "Country/Area" column to keep consistency between data frames. We reset the index value from 0 by sorting the "Country/Area" column in each dataset
        dataset.sort_values(by=['instance_id'], inplace=True)

        # reset index values so dataframes can be merged without errors
        data_to_be_merged = dataset.reset_index(drop=True)

        # 4. finially, merged datasets.
        if merged_all.empty:
            merged_all = data_to_be_merged
        else:
            merged_all = pd.merge(merged_all, data_to_be_merged, how='outer')

    return merged_all
# type(rock)


# check the shapes and if there is any missing values
def print_NA_details(dataset):
    '''method for checking whether the passed in dataset has missing (i.e. NA) fetatures '''
    missing = dataset.isnull().sum()
    missing = missing[missing > 0]
    print('\nmissing feature and values:\n', missing)


merged_dataset = merge_multi_datasets()
merged_dataset.to_csv("datasets/merged_all.csv", encoding='utf-8', index=False)

print(merged_dataset.info())
print(merged_dataset.shape)
print_NA_details(merged_dataset)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   instance_id       50000 non-null  int64  
 1   artist_name       50000 non-null  object 
 2   track_hash        50000 non-null  object 
 3   track_name        50000 non-null  object 
 4   popularity        50000 non-null  int64  
 5   acousticness      50000 non-null  float64
 6   danceability      50000 non-null  float64
 7   duration_ms       50000 non-null  int64  
 8   energy            50000 non-null  float64
 9   instrumentalness  50000 non-null  float64
 10  key               50000 non-null  object 
 11  liveness          50000 non-null  float64
 12  loudness          50000 non-null  float64
 13  mode              50000 non-null  object 
 14  speechiness       50000 non-null  float64
 15  tempo             50000 non-null  object 
 16  obtained_date     50000 non-null  object